### Problem: Maximize f(x) = x<sup>2</sup> - 11x + 150 when x = 0 to 255

Representation or ecoding solution:
x can be represented using 8 bits binary number.

# Step 1: Initialize Population

In [1]:
import random
def initPopulation(b=8, n =10): # n is the number of solutions, b is the length
    p = [random.choices([0,1],k = b) for i in range(n)] 
    return p

In [2]:
initPopulation(5,6)


[[0, 1, 1, 0, 0],
 [1, 0, 0, 1, 1],
 [1, 0, 1, 1, 0],
 [1, 0, 1, 0, 1],
 [1, 0, 0, 0, 0],
 [0, 1, 0, 0, 1]]

# Step 2: Reproduction or Parent Selection

Fitness Calculation

In [110]:
def binToDecimal(binList):
  decValue = 0 # decimel value
  power = len(binList)
  for digit in binList: # binary to decimel
    decValue += digit*2**(power-1)
    power -= 1
  return decValue

def func(x):
  return x**3 - 11*x**2 + 150

def getFitness(p): # p is the whole population
    f = [] # to store fitness values
    for sol in p:
        d = binToDecimal(sol)
        v = func(d)
        if v< 0: f.append(0)
        else: f.append(v) # fitness function
    return f

In [4]:
binToDecimal([1,0,0,1,0])

18

In [5]:
population = initPopulation(8,6)
getFitness(population)

[7092, 2570, 1176, 4410, 14400, 19026]

calculate probability and select parents

In [6]:
import numpy as np
def select_parent(n, pop): # probability, parent selection
    # probability
    fitness = getFitness(pop)
    total_fitness = sum(fitness)
    prob = [f/total_fitness for f in fitness]
    cumSumProb = np.cumsum(prob)
    #print(cumSumProb)
    # parent selection
    parents = []
    for i in range(n):
        # roullete wheel, generate a random number
        r = random.random()
        #print(r)
        # check bin number of r        
        for j in range(len(cumSumProb)):
            if r <= cumSumProb[j] :#checking the bin
                #print(j)
                parents.append(pop[j])
                break
    return parents

In [7]:
population = initPopulation(8,10)
print(population)
parents = select_parent(4, population)
print(parents)

[[0, 0, 1, 1, 0, 1, 0, 0], [0, 1, 1, 1, 1, 0, 0, 0], [1, 0, 0, 1, 1, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0, 0], [1, 0, 0, 1, 1, 0, 0, 1], [1, 0, 0, 1, 0, 0, 1, 0], [0, 0, 1, 0, 1, 0, 0, 1], [1, 1, 0, 1, 1, 1, 1, 0], [1, 1, 0, 0, 1, 1, 1, 0], [0, 1, 0, 1, 1, 1, 1, 1]]
[[1, 1, 0, 1, 1, 1, 1, 0], [1, 0, 0, 1, 1, 0, 0, 0], [1, 1, 0, 1, 1, 1, 1, 0], [1, 0, 0, 1, 1, 0, 0, 1]]


# Step 3: Crossover

In [59]:
# define a method crossover (input is parents set)
def cross(p):
  # print(p)
  num_cross = len(p)//2
  # print(num_cross)
  offsprings = []
  #  for loop to perform crossovers
  for i in range(num_cross):
   cp = random.choice(range(3,6))
   # print(cp)
   # perform crossover
   x,y = p[i],p[-(i+1)]
   # modify the above line as the parents will be different for each crossover
   off1 = x[:cp] + y[cp:]
   offsprings.append(off1)

   # write code for off2
   off2 = y[:cp] + x[cp:]
   offsprings.append(off2)

  return offsprings

In [41]:
off = cross(parents)
off

[[1, 1, 0, 0, 0, 1, 0, 1], [1, 1, 1, 1, 0, 1, 0, 1], [1, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 0, 0, 0, 0], [1, 1, 1, 1, 0, 1, 0, 1], [1, 1, 1, 1, 0, 0, 0, 0]]
3
4
5


[[1, 1, 0, 1, 0, 0, 0, 0],
 [1, 1, 1, 0, 0, 1, 0, 1],
 [1, 1, 1, 1, 0, 1, 0, 1],
 [1, 1, 1, 1, 0, 1, 0, 1],
 [1, 1, 1, 1, 0, 0, 0, 0],
 [1, 1, 1, 1, 0, 0, 0, 0]]

# Step 4: Mutation

In [142]:
# Perform mutation with 3% probability
def mutate(offsp, pr = 0.03):
  mut_offsp = offsp
  for i in range(0, len(offsp)):
    for j in range(0, len(offsp[i])):
      pr_r = random.choice(range(0, 100)) / 100
      if pr_r <= pr:
        # print("mutation at: ", i, " ", j)
        mut_offsp[i][j] = abs(mut_offsp[i][j] - 1)
  return mut_offsp

mutate(off)

[[1, 1, 0, 1, 0, 0, 1, 0],
 [1, 1, 1, 0, 0, 1, 0, 1],
 [1, 1, 0, 1, 1, 1, 1, 0],
 [1, 1, 1, 1, 1, 1, 0, 1],
 [1, 0, 0, 1, 0, 0, 0, 1],
 [1, 1, 1, 1, 0, 0, 0, 1]]

# Step 5: Select Survivor
The selected survivors (solutions) will be added to population

Select top 2 offspring based on fitness value and add them to population.

In [11]:
# define a method to select top offsprings (input offsprings, top)
def select_survivor(offsp):
  topOffsp = []
  f = getFitness(offsp)
  mInd = f.index(max(f))
  topOffsp.append(offsp[mInd])
  f.pop(mInd)
  #print(f)
  mInd = f.index(max(f))
  topOffsp.append(offsp[mInd])
  return topOffsp

In [12]:
select_survivor(off)

[[1, 1, 0, 1, 1, 0, 0, 0], [1, 1, 0, 1, 1, 0, 0, 0]]

In [13]:
def best(p):
  f = getFitness(p)
  mInd = f.index(max(f))
  return p[mInd], max(f)

# **Complete iteration**

In [145]:
# modify the below code to iterate the process for 3 generations
# in each generation show the best solution in population with fitness value
population = initPopulation(10,20)
# print(population)
print(best(population))

i = 1
while(True):
  parents = select_parent(4, population)
  offsprings = cross(parents)
  # call mutation here
  offsprings = mutate(offsprings, 0.03)
  survivors = select_survivor(offsprings)
  population = population + survivors
  # print(population)
  best_offspr = best(population)
  # print(best_offspr)
  i += 1
  if best_offspr[1] > 1059080000:
    print(i)
    print(best_offspr)
    break


([1, 1, 0, 1, 1, 1, 0, 1, 1, 0], 686871650)
52
([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 1059087498)


In [122]:
func(2**10 - 1)

1059087498